In [15]:
!pip install primisai openai nest-asyncio

import os
import nest_asyncio
from primisai.nexus.core import AI, Agent, Supervisor
from primisai.nexus.utils.debugger import Debugger
import json

nest_asyncio.apply()

In [16]:
print("🚀 PrimisAI Nexus Advanced Tutorial with OpenAI API")
print("=" * 55)

os.environ["OPENAI_API_KEY"] = "Use Your Own API Key Here5"

# llm_config = {
#     "api_key": os.environ["OPENAI_API_KEY"],
#     "model": "gpt-4o-mini",
#     "base_url": "https://api.openai.com/v1",
#     "temperature": 0.7
# }


llm_config = {
    "api_key": os.environ["OPENAI_API_KEY"],
    "model": "gpt-3.5-turbo",
    "base_url": "https://api.openai.com/v1",
    "temperature": 0.7
}


print("📋 API Configuration:")
print(f"• Model: {llm_config['model']}")
print(f"• Base URL: {llm_config['base_url']}")
print("• Note: OpenAI has limited free tokens through April 2025")
print("• Alternative: Consider Puter.js for unlimited free access")

🚀 PrimisAI Nexus Advanced Tutorial with OpenAI API
📋 API Configuration:
• Model: gpt-3.5-turbo
• Base URL: https://api.openai.com/v1
• Note: OpenAI has limited free tokens through April 2025
• Alternative: Consider Puter.js for unlimited free access


In [17]:
code_schema = {
    "type": "object",
    "properties": {
        "description": {"type": "string", "description": "Code explanation"},
        "code": {"type": "string", "description": "Python code implementation"},
        "language": {"type": "string", "description": "Programming language"},
        "complexity": {"type": "string", "enum": ["beginner", "intermediate", "advanced"]},
        "test_cases": {"type": "array", "items": {"type": "string"}, "description": "Example usage"}
    },
    "required": ["description", "code", "language"]
}

analysis_schema = {
    "type": "object",
    "properties": {
        "summary": {"type": "string", "description": "Brief analysis summary"},
        "insights": {"type": "array", "items": {"type": "string"}, "description": "Key insights"},
        "recommendations": {"type": "array", "items": {"type": "string"}, "description": "Action items"},
        "confidence": {"type": "number", "minimum": 0, "maximum": 1},
        "methodology": {"type": "string", "description": "Analysis approach used"}
    },
    "required": ["summary", "insights", "confidence"]
}

planning_schema = {
    "type": "object",
    "properties": {
        "tasks": {"type": "array", "items": {"type": "string"}, "description": "List of tasks to complete"},
        "priority": {"type": "string", "enum": ["low", "medium", "high"]},
        "estimated_time": {"type": "string", "description": "Time estimate"},
        "dependencies": {"type": "array", "items": {"type": "string"}, "description": "Task dependencies"}
    },
    "required": ["tasks", "priority"]
}

In [18]:
def calculate_metrics(data_str):
    """Calculate comprehensive statistics for numerical data"""
    try:
        data = json.loads(data_str) if isinstance(data_str, str) else data_str
        if isinstance(data, list) and all(isinstance(x, (int, float)) for x in data):
            import statistics
            return {
                "mean": statistics.mean(data),
                "median": statistics.median(data),
                "mode": statistics.mode(data) if len(set(data)) < len(data) else "No mode",
                "std_dev": statistics.stdev(data) if len(data) > 1 else 0,
                "max": max(data),
                "min": min(data),
                "count": len(data),
                "sum": sum(data)
            }
        return {"error": "Invalid data format - expecting array of numbers"}
    except Exception as e:
        return {"error": f"Could not parse data: {str(e)}"}

def validate_code(code):
    """Advanced code validation with syntax and basic security checks"""
    try:
        dangerous_imports = ['os', 'subprocess', 'eval', 'exec', '__import__']
        security_warnings = []

        for danger in dangerous_imports:
            if danger in code:
                security_warnings.append(f"Potentially dangerous: {danger}")

        compile(code, '<string>', 'exec')

        return {
            "valid": True,
            "message": "Code syntax is valid",
            "security_warnings": security_warnings,
            "lines": len(code.split('\n'))
        }
    except SyntaxError as e:
        return {
            "valid": False,
            "message": f"Syntax error: {e}",
            "line": getattr(e, 'lineno', 'unknown'),
            "security_warnings": []
        }

def search_documentation(query):
    """Simulate searching documentation (placeholder function)"""
    docs = {
        "python": "Python is a high-level programming language",
        "list": "Lists are ordered, mutable collections in Python",
        "function": "Functions are reusable blocks of code",
        "class": "Classes define objects with attributes and methods"
    }

    results = []
    for key, value in docs.items():
        if query.lower() in key.lower():
            results.append(f"{key}: {value}")

    return {
        "query": query,
        "results": results if results else ["No documentation found"],
        "total_results": len(results)
    }

In [19]:
print("\n📋 Setting up Multi-Agent Hierarchy with OpenAI")

main_supervisor = Supervisor(
    name="ProjectManager",
    llm_config=llm_config,
    system_message="You are a senior project manager coordinating development and analysis tasks. Delegate appropriately, provide clear summaries, and ensure quality delivery. Always consider time estimates and dependencies."
)

dev_supervisor = Supervisor(
    name="DevManager",
    llm_config=llm_config,
    is_assistant=True,
    system_message="You manage development tasks. Coordinate between coding, testing, and code review. Ensure best practices and security."
)

analysis_supervisor = Supervisor(
    name="AnalysisManager",
    llm_config=llm_config,
    is_assistant=True,
    system_message="You manage data analysis and research tasks. Ensure thorough analysis, statistical rigor, and actionable insights."
)

qa_supervisor = Supervisor(
    name="QAManager",
    llm_config=llm_config,
    is_assistant=True,
    system_message="You manage quality assurance and testing. Ensure thorough validation and documentation."
)


📋 Setting up Multi-Agent Hierarchy with OpenAI


In [20]:
code_agent = Agent(
    name="CodeWriter",
    llm_config=llm_config,
    system_message="You are an expert Python developer. Write clean, efficient, well-documented code with proper error handling. Always include test cases and follow PEP 8 standards.",
    output_schema=code_schema,
    tools=[{
        "metadata": {
            "function": {
                "name": "validate_code",
                "description": "Validates Python code syntax and checks for security issues",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "code": {"type": "string", "description": "Python code to validate"}
                    },
                    "required": ["code"]
                }
            }
        },
        "tool": validate_code
    }, {
        "metadata": {
            "function": {
                "name": "search_documentation",
                "description": "Search for programming documentation and examples",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Documentation topic to search for"}
                    },
                    "required": ["query"]
                }
            }
        },
        "tool": search_documentation
    }],
    use_tools=True
)

review_agent = Agent(
    name="CodeReviewer",
    llm_config=llm_config,
    system_message="You are a senior code reviewer. Analyze code for best practices, efficiency, security, maintainability, and potential issues. Provide constructive feedback and suggestions.",
    keep_history=True,
    tools=[{
        "metadata": {
            "function": {
                "name": "validate_code",
                "description": "Validates code syntax and security",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "code": {"type": "string", "description": "Code to validate"}
                    },
                    "required": ["code"]
                }
            }
        },
        "tool": validate_code
    }],
    use_tools=True
)

analyst_agent = Agent(
    name="DataAnalyst",
    llm_config=llm_config,
    system_message="You are a data scientist specializing in statistical analysis and insights generation. Provide thorough analysis with confidence metrics and actionable recommendations.",
    output_schema=analysis_schema,
    tools=[{
        "metadata": {
            "function": {
                "name": "calculate_metrics",
                "description": "Calculates comprehensive statistics for numerical data",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_str": {"type": "string", "description": "JSON string of numerical data array"}
                    },
                    "required": ["data_str"]
                }
            }
        },
        "tool": calculate_metrics
    }],
    use_tools=True
)

planner_agent = Agent(
    name="ProjectPlanner",
    llm_config=llm_config,
    system_message="You are a project planning specialist. Break down complex projects into manageable tasks with realistic time estimates and clear dependencies.",
    output_schema=planning_schema
)

tester_agent = Agent(
    name="QATester",
    llm_config=llm_config,
    system_message="You are a QA specialist focused on comprehensive testing strategies, edge cases, and quality assurance.",
    tools=[{
        "metadata": {
            "function": {
                "name": "validate_code",
                "description": "Validates code for testing",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "code": {"type": "string", "description": "Code to test"}
                    },
                    "required": ["code"]
                }
            }
        },
        "tool": validate_code
    }],
    use_tools=True
)

In [21]:
dev_supervisor.register_agent(code_agent)
dev_supervisor.register_agent(review_agent)
analysis_supervisor.register_agent(analyst_agent)
qa_supervisor.register_agent(tester_agent)

main_supervisor.register_agent(dev_supervisor)
main_supervisor.register_agent(analysis_supervisor)
main_supervisor.register_agent(qa_supervisor)
main_supervisor.register_agent(planner_agent)

In [22]:
print("\n🌳 Agent Hierarchy:")
main_supervisor.display_agent_graph()

print("\n🧪 Testing Full Multi-Agent Communication")
print("-" * 45)

try:
    test_response = main_supervisor.chat("Hello! Please introduce your team and explain how you coordinate complex projects.")
    print(f"✅ Supervisor communication test successful!")
    print(f"Response preview: {test_response[:200]}...")
except Exception as e:
    print(f"❌ Supervisor test failed: {str(e)}")
    print("Falling back to direct agent testing...")


🌳 Agent Hierarchy:
Main Supervisor: ProjectManager
│
├── Assistant Supervisor: DevManager
│   ├── Agent: CodeWriter
│   │   ├── Tool: validate_code
│   │   └── Tool: search_documentation
│   │
│   └── Agent: CodeReviewer
│       └── Tool: validate_code
│
├── Assistant Supervisor: AnalysisManager
│   └── Agent: DataAnalyst
│       └── Tool: calculate_metrics
│
├── Assistant Supervisor: QAManager
│   └── Agent: QATester
│       └── Tool: validate_code
│
└── Agent: ProjectPlanner
    └── No tools available

🧪 Testing Full Multi-Agent Communication
---------------------------------------------
❌ Supervisor test failed: Error in processing user input: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Falling b

In [23]:
print("\n🎯 Complex Multi-Agent Task Execution")
print("-" * 40)

complex_task = """Create a Python function that implements a binary search algorithm,
have it reviewed for optimization, tested thoroughly, and provide a project plan
for integrating it into a larger search system."""

print(f"Complex Task: {complex_task}")

try:
    complex_response = main_supervisor.chat(complex_task)
    print(f"✅ Complex task completed")
    print(f"Response: {complex_response[:300]}...")
except Exception as e:
    print(f"❌ Complex task failed: {str(e)}")


🎯 Complex Multi-Agent Task Execution
----------------------------------------
Complex Task: Create a Python function that implements a binary search algorithm, 
have it reviewed for optimization, tested thoroughly, and provide a project plan 
for integrating it into a larger search system.
❌ Complex task failed: Error in processing user input: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [24]:
print("\n🔧 Tool Integration & Structured Outputs")
print("-" * 43)

print("Testing Code Agent with tools...")
try:
    code_response = code_agent.chat("Create a function to calculate fibonacci numbers with memoization")
    print(f"✅ Code Agent with tools: Working")
    print(f"Response type: {type(code_response)}")

    if isinstance(code_response, str) and code_response.strip().startswith('{'):
        code_data = json.loads(code_response)
        print(f"  - Description: {code_data.get('description', 'N/A')[:50]}...")
        print(f"  - Language: {code_data.get('language', 'N/A')}")
        print(f"  - Complexity: {code_data.get('complexity', 'N/A')}")
    else:
        print(f"  - Raw response: {code_response[:100]}...")

except Exception as e:
    print(f"❌ Code Agent error: {str(e)}")

print("\nTesting Analyst Agent with tools...")
try:
    analysis_response = analyst_agent.chat("Analyze this sales data: [100, 150, 120, 180, 200, 175, 160, 190, 220, 185]. What trends do you see?")
    print(f"✅ Analyst Agent with tools: Working")

    if isinstance(analysis_response, str) and analysis_response.strip().startswith('{'):
        analysis_data = json.loads(analysis_response)
        print(f"  - Summary: {analysis_data.get('summary', 'N/A')[:50]}...")
        print(f"  - Confidence: {analysis_data.get('confidence', 'N/A')}")
        print(f"  - Insights count: {len(analysis_data.get('insights', []))}")
    else:
        print(f"  - Raw response: {analysis_response[:100]}...")

except Exception as e:
    print(f"❌ Analyst Agent error: {str(e)}")


🔧 Tool Integration & Structured Outputs
-------------------------------------------
Testing Code Agent with tools...
❌ Code Agent error: Error in chat processing: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Testing Analyst Agent with tools...
❌ Analyst Agent error: Error in chat processing: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [25]:
print("\n🔨 Manual Tool Usage")
print("-" * 22)

# Test all tools manually
sample_data = "[95, 87, 92, 88, 91, 89, 94, 90, 86, 93]"
metrics_result = calculate_metrics(sample_data)
print(f"Statistics for {sample_data}:")
for key, value in metrics_result.items():
    print(f"  {key}: {value}")

print("\nCode validation test:")
test_code = """
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1
"""
validation_result = validate_code(test_code)
print(f"Validation result: {validation_result}")

print("\nDocumentation search test:")
doc_result = search_documentation("python function")
print(f"Search results: {doc_result}")


🔨 Manual Tool Usage
----------------------
Statistics for [95, 87, 92, 88, 91, 89, 94, 90, 86, 93]:
  mean: 90.5
  median: 90.5
  mode: No mode
  std_dev: 3.0276503540974917
  max: 95
  min: 86
  count: 10
  sum: 905

Code validation test:
Validation result: {'valid': True, 'message': 'Code syntax is valid', 'security_warnings': [], 'lines': 13}

Documentation search test:
Search results: {'query': 'python function', 'results': ['No documentation found'], 'total_results': 0}


In [26]:
print("\n🚀 Advanced Multi-Agent Workflow")
print("-" * 35)

workflow_stages = [
    ("Planning", "Create a project plan for building a web scraper for news articles"),
    ("Development", "Implement the web scraper with error handling and rate limiting"),
    ("Review", "Review the web scraper code for security and efficiency"),
    ("Testing", "Create comprehensive test cases for the web scraper"),
    ("Analysis", "Analyze sample scraped data: [45, 67, 23, 89, 12, 56, 78, 34, 91, 43]")
]

workflow_results = {}

for stage, task in workflow_stages:
    print(f"\n{stage} Stage: {task}")
    try:
        if stage == "Planning":
            response = planner_agent.chat(task)
        elif stage == "Development":
            response = code_agent.chat(task)
        elif stage == "Review":
            response = review_agent.chat(task)
        elif stage == "Testing":
            response = tester_agent.chat(task)
        elif stage == "Analysis":
            response = analyst_agent.chat(task)

        workflow_results[stage] = response
        print(f"✅ {stage} completed: {response[:80]}...")

    except Exception as e:
        print(f"❌ {stage} failed: {str(e)}")
        workflow_results[stage] = f"Error: {str(e)}"


🚀 Advanced Multi-Agent Workflow
-----------------------------------

Planning Stage: Create a project plan for building a web scraper for news articles
❌ Planning failed: Error in chat processing: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Development Stage: Implement the web scraper with error handling and rate limiting
❌ Development failed: Error in chat processing: Chat completion failed: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [27]:
print("\n📊 System Monitoring & Performance")
print("-" * 37)

debugger = Debugger(name="OpenAITutorialDebugger")
debugger.log("Advanced OpenAI tutorial execution completed successfully")

print(f"Main Supervisor ID: {main_supervisor.workflow_id}")


📊 System Monitoring & Performance
-------------------------------------
Main Supervisor ID: 0d0b2bc3-b77e-4381-a647-f372f5709dee
